# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Векторизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логическая-регрессия" data-toc-modified-id="Логическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логическая регрессия</a></span></li><li><span><a href="#Классификатор-случайного-леса" data-toc-modified-id="Классификатор-случайного-леса-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Классификатор случайного леса</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

In [2]:
import pandas as pd

import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer 
import re 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv')

Откуда-то, абсолютно непонятно откуда взялся столбец Unnamed: 0?

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data = data.drop(['Unnamed: 0'], axis=1)

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

В нашей задаче наблюдается дисбаланс класссов, что может помешать в дальнейшем обучению. При обучении будем это учитывать.


### Лемматизация

Для лемматизации будем использовать Wordnet из NLTK

In [9]:
lemmatizer = WordNetLemmatizer()

In [10]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
def lemm_clear_text(text):
    reg = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = reg.split() 
    lemm = []
    for i in range(len(clear)):
        lemm.append(lemmatizer.lemmatize(clear[i], get_wordnet_pos(clear[i])))
    return " ".join(lemm)

In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lemm_clear_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


In [14]:
data = data.drop(['text'], axis=1)

Разделим данные на обучающую и тестовую выборки в отношении *75:25*.

In [15]:
target = data['toxic']
features = data['lemm_text']


features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=12345) 

### Векторизация

In [16]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train)

In [17]:
tf_idf_test = count_tf_idf.transform(features_test)

In [18]:
tf_idf_train.shape

(143362, 149895)

In [19]:
tf_idf_test.shape

(15930, 149895)

**Вывод**: В рамках подготовки данных была выполнена лемматизация и векторизация данных, был обнаружен дисбаланс классов, который будет учтен при обучении.

## Обучение

Рассмотрим три модели: 
- LogisticRegression;
- RandomForestClassifier;
- CatBoostClassifier.

### Логическая регрессия 

In [25]:
%%time
lr_model = LogisticRegression(max_iter=1000)
hyperparams = [{'C':[0.1, 1, 10],   
                'class_weight':['balanced']}]
clf = GridSearchCV(lr_model, hyperparams, scoring='f1',cv=3, n_jobs=-1, verbose=2)
clf.fit(tf_idf_train, target_train)
print()
print("Лучшие параметры модели:")
print()
LR_best_params = clf.best_params_
print(LR_best_params)
print()
print('F1:', clf.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END .......................C=0.1, class_weight=balanced; total time=  11.9s
[CV] END .......................C=0.1, class_weight=balanced; total time=  12.0s
[CV] END .......................C=0.1, class_weight=balanced; total time=  12.6s
[CV] END .........................C=1, class_weight=balanced; total time=  44.0s
[CV] END .........................C=1, class_weight=balanced; total time=  49.1s
[CV] END .........................C=1, class_weight=balanced; total time=  37.7s
[CV] END ........................C=10, class_weight=balanced; total time= 2.1min
[CV] END ........................C=10, class_weight=balanced; total time= 2.0min
[CV] END ........................C=10, class_weight=balanced; total time= 1.7min

Лучшие параметры модели:

{'C': 10, 'class_weight': 'balanced'}

F1: 0.7559256259013057
CPU times: user 4min 41s, sys: 6min 19s, total: 11min 1s
Wall time: 11min 2s


In [32]:
features_train_l = features_train.copy()
target_train_l = target_train.copy()

In [33]:
%%time
pipeline_lr = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", LogisticRegression(max_iter=1000)),
    ]
)

CPU times: user 137 µs, sys: 10 µs, total: 147 µs
Wall time: 151 µs


In [34]:
hyperparams ={'clf__solver':['newton-cg', 'lbfgs', 'liblinear'],
              'clf__C':[0.1, 1, 10],   
                'clf__class_weight':['balanced']}

In [36]:
%%time
grid_lr = GridSearchCV(estimator=pipeline_lr,param_grid=hyperparams, scoring='f1',cv=3, n_jobs=-1, verbose=2,)
grid_lr.fit(features_train_l, target_train_l)
print()
print("Лучшие параметры модели:")
print()
LR_best_params = grid_lr.best_params_
print(LR_best_params)
print()
print('F1:', grid_lr.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=newton-cg; total time=  25.2s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=newton-cg; total time=  26.2s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=newton-cg; total time=  26.2s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=lbfgs; total time=  34.5s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=lbfgs; total time=  29.6s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=lbfgs; total time=  30.1s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=liblinear; total time=  13.6s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=liblinear; total time=  14.8s
[CV] END clf__C=0.1, clf__class_weight=balanced, clf__solver=liblinear; total time=  13.3s
[CV] END clf__C=1, clf__class_weight=balanced, clf__solver=newton-cg; total time=  41.3s
[CV] END clf__C=1, clf__class_weight=balance

###  Классификатор случайного леса

In [20]:
%%time
rf_model = RandomForestClassifier(random_state=12345)
hyperparams = [{'n_estimators':[100, 1000, 1500],   
                'class_weight':['balanced'],
               'max_depth': range(2,8)}]
rf = GridSearchCV(rf_model,
                  hyperparams,
                  scoring='f1',
                  cv=3,
                  n_jobs=-1,
                  verbose=2)
rf.fit(tf_idf_train, target_train)
print()
print("Лучшие параметры модели (случайный лес):")
print()
RF_best_params = rf.best_params_
print(RF_best_params)
print()
print('F1:', rf.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END class_weight=balanced, max_depth=2, n_estimators=100; total time=   4.8s
[CV] END class_weight=balanced, max_depth=2, n_estimators=100; total time=   4.7s
[CV] END class_weight=balanced, max_depth=2, n_estimators=100; total time=   4.5s
[CV] END class_weight=balanced, max_depth=2, n_estimators=1000; total time=  45.3s
[CV] END class_weight=balanced, max_depth=2, n_estimators=1000; total time=  47.6s
[CV] END class_weight=balanced, max_depth=2, n_estimators=1000; total time=  46.4s
[CV] END class_weight=balanced, max_depth=2, n_estimators=1500; total time= 1.2min
[CV] END class_weight=balanced, max_depth=2, n_estimators=1500; total time= 1.1min
[CV] END class_weight=balanced, max_depth=2, n_estimators=1500; total time= 1.1min
[CV] END class_weight=balanced, max_depth=3, n_estimators=100; total time=   6.5s
[CV] END class_weight=balanced, max_depth=3, n_estimators=100; total time=   6.2s
[CV] END class_weight=balanced,

### CatBoost

In [21]:
%%time

classificator = CatBoostClassifier(verbose=False, iterations=200)
classificator.fit(tf_idf_train, target_train)
cv_f1_CBC = cross_val_score(classificator,
                                         tf_idf_train, 
                                         target_train,
                                         scoring='f1').mean()
print('F1', cv_f1_CBC)

F1 0.7364657999679491
CPU times: user 43min 34s, sys: 26.6 s, total: 44min 1s
Wall time: 44min 10s


**Вывод**: Получили, что моделью с лучшим значением *F1= 0.762477* является LogisticRegression

## Выводы

**LogisticRegression**

In [39]:
model = LogisticRegression(C=10, class_weight= 'balanced',solver='newton-cg', max_iter=1000)
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
f1 = f1_score(target_test, predictions)
print('F1 регрессии:', f1)

F1 регрессии: 0.7500707613925841


## Итоговый вывод

В рамках проекта были выполнены следующие действия:
- была выполнена подготовка данных: был выявлен дисбаланс классов, были выполнены лемматизация и векторизация;
- были обучены 3 модели, и получено, что лучшей моделью по показателю метрики *F1=0.7652* является  LogisticRegression;
- получено значение *F1* на тестовой выборке для всех моделей, лучший показатель *F1=0.750* у LogisticRegression, что удовлетворяет поставленной цели.